<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/9e6101d95936f4bd4d390efc9ce646dc6937fb2d/images/socal/github-large-banner-phoenix.jpg" width="1000"/>
        <br>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tracing CrewAI with Arize Phoenix - Orchestrator Workers Workflow</h1>

In [1]:
%pip install -q arize-phoenix opentelemetry-sdk opentelemetry-exporter-otlp crewai crewai_tools openinference-instrumentation-crewai

Note: you may need to restart the kernel to use updated packages.


# Set up Keys and Dependencies

Note: For this colab you'll need:

*   OpenAI API key (https://openai.com/)
*   Serper API key (https://serper.dev/)
*   Phoenix API key (https://app.phoenix.arize.com/)

In [2]:
import getpass
import os

# Prompt the user for their API keys if they haven't been set
openai_key = os.getenv("OPENAI_API_KEY", "OPENAI_API_KEY")
serper_key = os.getenv("SERPER_API_KEY", "SERPER_API_KEY")

if openai_key == "OPENAI_API_KEY":
    openai_key = getpass.getpass("Please enter your OPENAI_API_KEY: ")

if serper_key == "SERPER_API_KEY":
    serper_key = getpass.getpass("Please enter your SERPER_API_KEY: ")

# Set the environment variables with the provided keys
os.environ["OPENAI_API_KEY"] = openai_key
os.environ["SERPER_API_KEY"] = serper_key

if "PHOENIX_API_KEY" not in os.environ:
    os.environ["PHOENIX_API_KEY"] = getpass.getpass("Enter your Phoenix API key: ")

os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={os.environ['PHOENIX_API_KEY']}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com/"

## Configure Tracing

In [23]:
from phoenix.otel import register

tracer_provider = register(
    project_name="crewai-agents", endpoint="https://app.phoenix.arize.com/v1/traces"
)

Overriding of current TracerProvider is not allowed


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: crewai-agents
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'api_key': '****', 'authorization': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



# Instrument CrewAI

In [24]:
from openinference.instrumentation.crewai import CrewAIInstrumentor

CrewAIInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)

Attempting to instrument while already instrumented


## Define your Agents

In [15]:
from crewai import Agent, Crew, Process, Task

# Define worker agents
trend_researcher = Agent(
    role="AI Trend Researcher",
    goal="Analyze current advancements in AI",
    backstory="Expert in tracking and analyzing new trends in artificial intelligence.",
    verbose=True,
)

policy_analyst = Agent(
    role="AI Policy Analyst",
    goal="Examine the implications of AI regulations and governance",
    backstory="Tracks AI policy developments across governments and organizations.",
    verbose=True,
)

risk_specialist = Agent(
    role="AI Risk Specialist",
    goal="Identify potential risks in frontier AI development",
    backstory="Focuses on safety, alignment, and misuse risks related to advanced AI.",
    verbose=True,
)

synthesizer = Agent(
    role="Synthesis Writer",
    goal="Summarize all findings into a final cohesive report",
    backstory="Expert at compiling research insights into executive-level narratives.",
    verbose=True,
)

orchestrator = Agent(
    role="Orchestrator",
    goal=(
        "Your job is to delegate research and writing tasks to the correct coworker using the 'Delegate work to coworker' tool.\n"
        "For each task you assign, you MUST call the tool with the following JSON input:\n\n"
        "{\n"
        '  "task": "Short summary of the task to do (plain string)",\n'
        '  "context": "Why this task is important or part of the report (plain string)",\n'
        '  "coworker": "One of: AI Trend Researcher, AI Policy Analyst, AI Risk Specialist, Synthesis Writer"\n'
        "}\n\n"
        "IMPORTANT:\n"
        "- Do NOT format 'task' or 'context' as dictionaries.\n"
        "- Do NOT include types or nested descriptions.\n"
        "- Only use plain strings for both.\n"
        "- Call the tool multiple times, one per coworker."
    ),
    backstory="You are responsible for assigning each part of an AI report to the right specialist.",
    verbose=True,
    allow_delegation=True,
)

## Define your Tasks

In [16]:
# Define the initial task only for the orchestrator
initial_task = Task(
    description="Create an AI trends report. It should include recent innovations, policy updates, and safety risks. Then synthesize it into a unified summary.",
    expected_output="Assign subtasks via the DelegateWorkTool and return a final report.",
    agent=orchestrator,
)

# Set up the crew (no hierarchical process needed with delegation tools)
crew = Crew(
    agents=[trend_researcher, policy_analyst, risk_specialist, synthesizer],
    tasks=[initial_task],
    manager_agent=orchestrator,
    verbose=True,
)

# Run the full workflow
result = crew.kickoff()
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 1951f90a-8d81-4060-aa0d-bf2fadb42ada                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 6dd8c7c8-3aeb-4d1d-a8c3-600c5730b8bc
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 6dd8c7c8-3aeb-4d1d-a8c3-600c5730b8bc
       Status: Executing Task...
    └── 🤖 Agent: Orchestrator
            Status: In Progress

# Agent: Orchestrator
## Task: Create an AI trends report. It should include recent innovations, policy updates, and safety risks. Then synthesize it into a unified summary.


🤖 Agent: Orchestrator
    Status: In Progress

...

🚀 Crew: crew
└── 📋 Task: 6dd8c7c8-3aeb-4d1d-a8c3-600c5730b8bc
       Status: Executing Task...
    ├── 🤖 Agent: Orchestrator
    │       Status: In Progress
    ├── 🤖 Agent: AI Trend Researcher
    │       Status: ✅ Completed
    ├── 🤖 Agent: AI Policy Analyst
    │       Status: ✅ Completed
    ├── 🤖 Agent: AI Risk Specialist
    │       Status: ✅ Completed
    └── 🤖 Agent: Orchestrator
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: 6dd8c7c8-3aeb-4d1d-a8c3-600c5730b8bc
       Assigned to: Orchestrator
       Status: ✅ Completed
    ├── 🤖 Agent: Orchestrator
    │       Status: In Progress
    ├── 🤖 Agent: AI Trend Researcher
    │       Status: ✅ Completed
    ├── 🤖 Agent: AI Policy Analyst
    │       Status: ✅ Completed
    ├── 🤖 Agent: AI Risk Specialist
    │       Status: ✅ Completed
    └── 🤖 Agent: Orchestrator
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6dd8c7c8-3aeb-4d1d-a8c3-600c5730b8bc                                                                     │
│  Agent: Orchestrator                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 1951f90a-8d81-4060-aa0d-bf2fadb42ada                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

**AI Innovations, Policy Updates, and Safety Risks: A Comprehensive Overview**

**Introduction**  
Recent advancements in artificial intelligence (AI) have accelerated at an unprecedented pace, driven by breakthroughs in machine learning, natural language processing, and computer vision. As these technologies evolve, understanding their implications is crucial for stakeholders across various sectors. This report synthesizes key AI innovations, recent policy updates, and associated safety risks, offering a unified perspective on the current landscape.

**Recent AI Innovations**  
1. **Generative AI Advances**: The emergence of generative AI models such as OpenAI's GPT-4 and Google's Bard has transformed content creation, enabling high-quality text, image, and video generation. These tools have broad applications, from marketing to entertainment, enhancing creativity while raising ethical considerations regarding authorship and misinformation.

2. **AI in Healthcare**: AI applications in

## Create Crew

In [11]:
# 👥 Assemble the Crew
crew = Crew(
    agents=[trend_researcher, policy_analyst, risk_specialist, synthesizer],
    tasks=[initial_task],
    manager_agent=orchestrator,
    process=Process.hierarchical,
    verbose=True,
)


# 🚀 Run the Crew
result = crew.kickoff()

# 🖨️ Display the result
if isinstance(result, list):
    for r in result:
        print(getattr(r, "text", str(r)))
else:
    print(getattr(result, "text", str(result)))

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 779f483a-6e73-4619-a089-4ec18d76f935                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: aaa746ba-6e96-4942-86b6-416ad97bd251
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: aaa746ba-6e96-4942-86b6-416ad97bd251
       Status: Executing Task...
    └── 🤖 Agent: Orchestrator
            Status: In Progress

# Agent: Orchestrator
## Task: Create an AI trends report that covers recent innovations, risk areas, and policy updates.
 

Action '```
{
  "task": "Research recent innovations in AI technology and provide insights.",
  "context": "This is part of the AI trends report which covers the latest breakthroughs and developments in AI.",
  "coworker": "AI Trend Researcher"
}
```

Action 2:
```
{
  "task": "Identify and analyze potential risk areas associated with AI technologies.",
  "context": "This is part of the AI trends report focused on understanding the risks that come with AI advancements.",
  "coworker": "AI Risk Specialist"
}
```

Action 3:
```
{
  "task": "Investigate recent policy updates and changes related to AI governance.",
  "context": "This will provide essential context for the AI trends report regarding regulations and policies affecting AI.",
  "coworker": "AI Policy Analyst"
}
```

I will now execute these actions one at a time and gather the results.

```
Thought: I wi

🤖 Agent: Orchestrator
    Status: In Progress

...

🚀 Crew: crew
└── 📋 Task: aaa746ba-6e96-4942-86b6-416ad97bd251
       Status: Executing Task...
    └── 🤖 Agent: Orchestrator
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: aaa746ba-6e96-4942-86b6-416ad97bd251
       Assigned to: Orchestrator
       Status: ✅ Completed
    └── 🤖 Agent: Orchestrator
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: aaa746ba-6e96-4942-86b6-416ad97bd251                                                                     │
│  Agent: Orchestrator                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 779f483a-6e73-4619-a089-4ec18d76f935                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

[
    {
        "subtask": "Research recent innovations in AI technology and provide insights.",
        "assigned_to": "AI Trend Researcher"
    },
    {
        "subtask": "Identify and analyze potential risk areas associated with AI technologies.",
        "assigned_to": "AI Risk Specialist"
    },
    {
        "subtask": "Investigate recent policy updates and changes related to AI governance.",
        "assigned_to": "AI Policy Analyst"
    },
    {
        "subtask": "Compile the report based on the gathered information.",
        "assigned_to": "Orchestrator"
    }
]


### Check your Phoenix project to view the traces and spans from your runs.